# Data Collection

In [2]:
!pip install nbimporter wrds

You should consider upgrading via the 'c:\users\rjsmi\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [3]:
# import nbimporter
from secret.credentials import WRDS_USERNAME, WRDS_PASS
import pandas as pd
import wrds

# Stock datatype, sectors is a list of tuples with S&P sector information
from sector import Stock, sectors

In [26]:
# Authenticate and cache credentials
db = wrds.Connection(wrds_username=WRDS_USERNAME)

Loading library list...
Done


In [27]:
START_DT = '2015-01-01'

sp5 = Stock('SP500', '', 'S&P 500 Index')
df = db.raw_sql("SELECT 'SP500' as ticker, 'S&P 500 Index' as descr, caldt as date,spindx as close,sprtrn as retx FROM crsp.dsp500 WHERE caldt > '{}'".format(START_DT))
df.index = df["date"]
df.drop(columns=["date"], inplace=True)
sp5.data = df
# Can use the get_table function, however this is limited on our SQL input 
# data = db.get_table(library='crsp', table='dsp500', columns = ['caldt', 'spindx', 'sprtrn'])

stock_data = [sp5]

for desc, ticker, cusip in sectors:
    sector = Stock(ticker, cusip, desc)
    df = db.raw_sql("SELECT '{}' as ticker, '{}' as descr, date, bidlo as low, askhi as high, ABS(prc) as close, vol, ret, bid, ask, retx FROM crsp.dsf WHERE cusip = '{}' AND date > '{}'".format(ticker, desc, cusip, START_DT))
    df.index = df["date"]
    df.drop(columns=["date"], inplace=True)
    sector.data = df
    stock_data.append(sector)


In [28]:
# db.list_tables(library="crsp")
# db.describe_table('crsp', 'msf')

In [30]:
# Terminate our connection for data gathering
db.close()

In [31]:
# Compute HIGH, LOW from data (needed for technical analysis)

all_data = []
for stock in stock_data:
    all_data.append(stock.data)

joined = pd.concat(all_data)

In [32]:
# Save our data
for stock in stock_data:
    stock.data.to_csv("data/sectors/{}_7yr_daily.csv".format(stock.ticker))

In [23]:
joined

,ticker,descr,close,retx,low,high,vol,ret,bid,ask
date,,,,,,,,,,
2015-01-02,SP500,S&P 500 Index,2058.200000,-0.000340,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-05,SP500,S&P 500 Index,2020.580000,-0.018278,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-06,SP500,S&P 500 Index,2002.610000,-0.008893,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-07,SP500,S&P 500 Index,2025.900000,0.011630,NaN,NaN,NaN,NaN,NaN,NaN
2015-01-08,SP500,S&P 500 Index,2062.140000,0.017888,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2021-12-27,XLU,Utilities,70.059998,0.004877,69.540001,70.059998,7853685.0,0.004877,70.050003,70.059998
2021-12-28,XLU,Utilities,70.690002,0.008992,70.110001,70.709999,8052724.0,0.008992,70.699997,70.709999
2021-12-29,XLU,Utilities,71.070000,0.005376,70.650002,71.180000,6877849.0,0.005376,71.059998,71.070000


In [24]:
joined.to_csv("data/sp5_sectors_7yr_daily.csv")